In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt
import numpy as np

In [7]:
# Load and concatenate the CSV files from 2000 to 2024
years = list(range(2000, 2025))
dfs = []

# Assuming all the CSV files are in the format 'daily_aqi_by_county_<year>.csv'
for year in years:
    file_name = f"../daily_aqi_data/daily_aqi_by_county_{year}.csv"
    df = pd.read_csv(file_name)
    dfs.append(df)

print(df.head())

/var/folders/n8/089mkp9j15jc90lqvrpw8yym0000gn/T/ipykernel_29403/233154234.py:8: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_name)
/var/folders/n8/089mkp9j15jc90lqvrpw8yym0000gn/T/ipykernel_29403/233154234.py:8: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_name)
/var/folders/n8/089mkp9j15jc90lqvrpw8yym0000gn/T/ipykernel_29403/233154234.py:8: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_name)
/var/folders/n8/089mkp9j15jc90lqvrpw8yym0000gn/T/ipykernel_29403/233154234.py:8: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_name)
/var/folders/n8/089mkp9j15jc90lqvrpw8yym0000gn/T/ipykernel_29403/233154234.py:8: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import 

  State Name county Name  State Code  County Code        Date  AQI Category  \
0    Alabama     Baldwin           1            3  2024-01-03   41     Good   
1    Alabama     Baldwin           1            3  2024-01-04   38     Good   
2    Alabama     Baldwin           1            3  2024-01-05   44     Good   
3    Alabama     Baldwin           1            3  2024-01-06    7     Good   
4    Alabama     Baldwin           1            3  2024-01-07   29     Good   

  Defining Parameter Defining Site  Number of Sites Reporting  
0              PM2.5   01-003-0010                          1  
1              PM2.5   01-003-0010                          1  
2              PM2.5   01-003-0010                          1  
3              PM2.5   01-003-0010                          1  
4              PM2.5   01-003-0010                          1  


In [17]:
# Concatenate all years' data into one DataFrame
data = pd.concat(dfs, ignore_index=True)

# Convert 'Date' column to datetime format and extract 'Year'
data["Date"] = pd.to_datetime(data["Date"])
data["Year"] = data["Date"].dt.year

# Group by state name and year, and calculate average AQI and most common category
state_yearly_aqi = (
    data.groupby(["State Name", "Year"])
    .agg(
        AQI=("AQI", "mean"),
        Category=("Category", lambda x: x.mode()[0] if not x.mode().empty else None),
    )
    .reset_index()
)

state_yearly_aqi.head()

,State Name,Year,AQI,Category
0,Alabama,2000,64.233499,Moderate
1,Alabama,2001,55.147858,Good
2,Alabama,2002,52.424248,Good
3,Alabama,2003,51.357906,Good
4,Alabama,2004,49.689020,Good


In [28]:
# Create an interactive bar chart for AQI by state for a specific year
# Filter out Mexico from the data
state_yearly_aqi = state_yearly_aqi[
    state_yearly_aqi["State Name"] != "Country Of Mexico"
]

# Define the year parameter with a slider selection
year_param = alt.param(
    name="Select_Year",
    bind=alt.binding_range(
        min=2000,
        max=2024,
        step=1,
        name="Year: ",
    ),
    value=2024,
)

bar_chart = (
    alt.Chart(state_yearly_aqi)
    .mark_bar()
    .encode(
        x=alt.X("State Name:N", sort="-y", title="State"),
        y=alt.Y("AQI:Q", title="Average AQI", scale=alt.Scale(domain=[0, 75])),
        color=alt.Color("AQI:Q", title="Average AQI"),
        tooltip=["State Name", "AQI", "Category"],
    )
    .add_params(year_param)
    .transform_filter(alt.datum.Year == year_param)
    .properties(title="Average AQI by State for Selected Year", width=700, height=400)
)

bar_chart

alt.Chart(...)